In [9]:
import pandas as pd

import spacy
from spacy import displacy

from spacy.matcher import Matcher
from spacy.matcher import DependencyMatcher
from spacy.tokens import Token

Token.set_extension("ignore", default=False, force=True)

nlp = spacy.load("de_dep_news_trf")

In [20]:
sentences = pd.read_csv('demo_data_real.csv')
sentence = sentences.iloc[1]['Original Text']
sentence

'Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.'

# General Preprocessing

In [11]:
sentences.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)

In [12]:
sentences

,ID,Bereinigter Text,Tags/Kategorie,Eingereicht von,Original Text
0,1,Die Tabelle “Subscriptions” im Schema “CRM” en...,"DWH, SQL",NaN,"Hey, du kannst die Tabelle “Subscriptions” im ..."
1,2,Maximal 3 Urlaubstage mit ins nächste Jahr neh...,HR,NaN,"Hallo Marcel, hier noch eine kurze Info für di..."
2,3,CR bedeutet Conversion Rate.,"Allgemeine Definitionen, CRM",NaN,"Hi Marcel, CR bedeutet in unserem Kontext Conv..."
3,4,AE bedeutet Account Executive.,"Allgemeine Definitionen, CRM",NaN,"AE = Account Executive, also Sales Mitarbeiter..."
4,5,Mit “Abrechnungsdatum” ist das Attribut “Purch...,"Allgemeine Definitionen, DWH",NaN,"Hi Marcel, ja genau mit “Abrechnungsdatum” ist..."
5,6,Die gebuchten Addons sind in der Tabelle “Subs...,"DWH, SQL",NaN,"Hey Marcel, die gebuchten Addons findest du in..."
6,7,"""Deactivated Date” steht für den Zeitpunkt, wa...","Allgemeine Definitionen, CRM",NaN,"Hi, das Attribut “Deactivated Date” steht für ..."
7,8,"Die wichtigsten Tabellen sind “Owners”, “Subsc...","CRM, DWH",NaN,"Hey Marcel, erstmal noch eine kurze Info für d..."
8,9,Mit dem Datenintegrationstool Stitch können wi...,DWH,NaN,"Hey Marcel, ich weiß nicht, ob du das Tool Sti..."
9,10,Als Werkstudent ist der Abrechnungszeitraum fü...,HR,NaN,"Hi Marcel, eine kurze Info noch für dich: Als ..."


# Generate Rules

In [23]:
doc = nlp(sentence)

In [24]:
displacy.serve(doc, style='dep')

c:\Users\jdhau\anaconda3\lib\site-packages\spacy\displacy\__init__.py:103: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [26]:
def set_ignore(matcher, doc, id, matches):
    for _, start, end in matches:
        for tok in doc[start:end]:
            tok._.ignore = True

# Grußformeln (Ende)

In [27]:
Grußformelmatcher = Matcher(nlp.vocab)
pattern = [
    [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "PROPN"}],
    [{"POS": "NOUN"}, {"POS": "PROPN"}],
    ]
Grußformelmatcher.add("Grußformeln", pattern, on_match=set_ignore)

In [28]:
matches = Grußformelmatcher(doc)

print(matches)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

[]


# Begrüßungsformeln

In [29]:
Begrüßungsformelmatcher = Matcher(nlp.vocab)

pattern = [{"ORTH": "Hallo"}, {"POS": "PROPN"}, {"IS_PUNCT": True}]
pattern = [{"ORTH": "Hallo"}, {"POS": "ADV"}, {"IS_PUNCT": True}]

Begrüßungsformelmatcher.add("Begrüßungformeln", [pattern], on_match=set_ignore) # ,  

In [30]:
matches = Begrüßungsformelmatcher(doc)

print(matches)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

[]


# Füllwortmatcher

In [31]:
Füllwortmatcher = Matcher(nlp.vocab)

pattern = [
    [{"lower": "erstmal"}],
    [{"lower": "erst"}, {"lower": "einmal"}],
    [{"lower": "ja"}, {"lower": "genau"}],
    [{"lower": "exakt"}],
    [{"lower": "jawohl"}],
    ]
Füllwortmatcher.add("Füllwörter", pattern, on_match=set_ignore)



In [32]:
matches = Füllwortmatcher(doc)

print(matches)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

[]


# Resultat

In [33]:
toks = [tok.text + tok.whitespace_ for tok in doc if not tok._.ignore]
print("".join(toks))

Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.


# Multiprocessing

In [43]:
docs = nlp.pipe(sentences['Original Text'], n_process=4)

In [44]:
for doc in docs:
    print(doc.text)
    print([(ent.text, ent.label_) for ent in doc.ents])

Hey, du kannst die Tabelle “Subscriptions” im Schema “CRM” benutzen. Dort sind alle Daten zu den Abonnements enthalten.
[]
Hallo Marcel, hier noch eine kurze Info für dich: Du kannst maximal 3 Urlaubstage mit ins nächste Jahr nehmen. Alle anderen Urlaubstage sollten bis Ende des Jahres verplant sein.
[]
Hi Marcel, CR bedeutet in unserem Kontext Conversion Rate, also z.B. CR overall Expansion = Conversion von erster Deal Stage bis Deal Stage “Closed”.
[]
AE = Account Executive, also Sales Mitarbeiter, die ab dem Zeitpunkt “Demo Scheduled” im Sales-Prozess übernehmen (vorherige Sales Stages: SD = Sales Development).
[]
Hi Marcel, ja genau mit “Abrechnungsdatum” ist das Attribut “Purchased At” auf Subsription Ebene gemeint.
[]
Hey Marcel, die gebuchten Addons findest du in der Tabelle “Subscriptions” unter dem Attribut “title”.
[]
Hi, das Attribut “Deactivated Date” steht für den Zeitpunkt, wann ein Kunde, der seinen Vertrag gekündigt hat, final bei uns im System deaktiviert wurde.
[]
Hey

# Categorization